# **General**

'''
Questo è un esempio di una lista Python contenente dati strutturati per oggetti 3D, come porte,
con informazioni su classi semantiche, affordance, e punti nello spazio 3D.
Ogni elemento della lista è un dizionario con le seguenti chiavi:

shape_id: Identificativo univoco della forma.
semantic class: Classe semantica dell'oggetto (ad esempio "Door").
affordance: Lista di funzionalità/azioni che l'oggetto può supportare, come "grasp", "openable", ecc.
full_shape: Un dizionario contenente:
coordinate: Array numpy di coordinate 3D dei punti che definiscono la forma.
label: Dizionario che mappa le affordance a valori numerici per ciascun punto (es. 0 per "non applicabile", valori >0 per un grado di applicabilità).
Se hai bisogno di manipolare questi dati o di estrarre specifiche informazioni, fammi sapere!

In [1]:
!pip install open3d

In [2]:
import pickle
import plotly.graph_objects as go
import numpy as np
import open3d as o3d

In [3]:
from google.colab import drive
drive.mount('/content/drive')

!cd content/drive/MyDrive/Affordance_Highlighting_Project_2024-main/AML\ Project:\ 3D\ Affordance/dataset/full-shape/ && ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/bin/bash: line 1: cd: content/drive/MyDrive/Affordance_Highlighting_Project_2024-main/AML Project: 3D Affordance/dataset/full-shape/: No such file or directory


# **Load Dataset**

In [4]:
def load_data(file_path):
    # Open and load file
    with open(file_path, "rb") as file:
        data = pickle.load(file)
    return data

In [5]:
# Path .pkl file
file_path = r"/content/drive/MyDrive/AML Project: 3D Affordance/dataset/full-shape/full_shape_train_data.pkl"
data = load_data(file_path)

**Visualizing available objects**

In [ ]:
def get_first_elements(dataset, n=1):
    """
    Ritorna i primi n elementi per ogni classe semantica nel dataset.

    Args:
        dataset (list): Lista di dizionari con la struttura fornita.
        n (int): Numero di elementi da restituire per ogni classe semantica.

    Returns:
        list: Una lista di tuple con la classe semantica e l'indice dell'elemento.
    """
    semantic_class_indices = []

    for idx, item in enumerate(dataset):
        semantic_class = item['semantic class']
        if len([x for x in semantic_class_indices if x[0] == semantic_class]) < n:
            semantic_class_indices.append((semantic_class, idx))

    return semantic_class_indices

result = get_first_elements(data, n=1)
for semantic_class, index in result:
    print(f"Semantic class: {semantic_class}, Index: {index}")

Semantic class: Door, Index: 0
Semantic class: Clock, Index: 154
Semantic class: Dishwasher, Index: 522
Semantic class: Earphone, Index: 639
Semantic class: Vase, Index: 796
Semantic class: Knife, Index: 1531
Semantic class: Bowl, Index: 1756
Semantic class: Bag, Index: 1888
Semantic class: Faucet, Index: 1976
Semantic class: Scissors, Index: 2417
Semantic class: Display, Index: 2466
Semantic class: Chair, Index: 3088
Semantic class: Bottle, Index: 7368
Semantic class: Microwave, Index: 7656
Semantic class: StorageFurniture, Index: 7786
Semantic class: Refrigerator, Index: 9317
Semantic class: Mug, Index: 9447
Semantic class: Keyboard, Index: 9580
Semantic class: Table, Index: 9690
Semantic class: Bed, Index: 15283
Semantic class: Hat, Index: 15410
Semantic class: Laptop, Index: 15566
Semantic class: TrashCan, Index: 15861


# **Select object and label**

In [6]:
selected_obj_index = 7368
selected_label = 'openable'

**Visualizing parts of dataset**

In [7]:
# Visualizza una parte del contenuto
if isinstance(data, dict):
    print("Chiavi del dizionario:", list(data.keys()))
elif isinstance(data, list):
    #print("Esempio di elementi nella lista:", data[selected_obj_index])
    # Estrazione delle caratteristiche
    shape_id = data[selected_obj_index]['shape_id']
    semantic_class = data[selected_obj_index]['semantic class']
    affordances = data[selected_obj_index]['affordance']
    coordinates = data[selected_obj_index]['full_shape']['coordinate']
    labels = data[selected_obj_index]['full_shape']['label'][selected_label]

    label_names = list(data[selected_obj_index]['full_shape']['label'].keys())

    # Output delle variabili
    #print("Shape ID:", shape_id)
    #print("Semantic Class:", semantic_class)
    #print("Affordances:", affordances)
    #print("Coordinates:", coordinates)
    print("Labels info:", data[selected_obj_index]['full_shape']['label'][selected_label])
    print(f"Available labels for {semantic_class} object (index: {selected_obj_index}):\n", label_names)
else:
    print("Contenuto:", data)

Labels info: [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
Available labels for Bottle object (index: 7368):
 ['grasp', 'contain', 'lift', 'openable', 'layable', 'sittable', 'support', 'wrap_grasp', 'pourable', 'move', 'displaY', 'pushable', 'pull', 'listen', 'wear', 'press', 'cut', 'stab']


# **Functions for visualizing Point Cloud/Mesh**

In [8]:
'''
  Function for visualizing Point Cloud with and without Highlighting
'''
def show_pc(pcd, labels = None):
          if labels is None:
            print("Point Cloud without Highlighting")
            fig = go.Figure(
                data=[
                  go.Scatter3d(
                    x=coordinates[:, 0],
                    y=coordinates[:, 1],
                    z=coordinates[:, 2],
                    mode='markers',
                    marker=dict(
                        size=3,
                        color=coordinates[:, 2],
                        colorscale='Viridis',
                        opacity=0.7)
                )])
          else:
            print("Point Cloud with Highlighting")
            fig = go.Figure(data=[
                go.Scatter3d(
                    x=coordinates[:, 0],
                    y=coordinates[:, 1],
                    z=coordinates[:, 2],
                    mode='markers',
                    marker=dict(
                        size=3,
                        color=labels,  # Colors based on label
                        colorscale='Viridis',
                        colorbar=dict(title='Density'),
                        opacity=0.7)
                )])


          fig.update_layout(
            title='Point Cloud' if labels is None else 'Point Cloud with Highlighting',
            scene=dict(
                xaxis=dict(showgrid=False, showticklabels=False, showbackground=False, title=''),
                yaxis=dict(showgrid=False, showticklabels=False, showbackground=False, title=''),
                zaxis=dict(showgrid=False, showticklabels=False, showbackground=False, title=''),
                aspectmode='data'
                )
          )

          fig.show()



'''
  Function for visualizing Mesh with and without Highlighting
'''
def show_mesh(mesh, labels = None):
  # Colors
  base_color = 'rgb(169, 169, 169)' #grey
  highlithed_color = 'rgb(255, 255, 0)' #yellow

  # Get info mesh
  vertices = np.asarray(mesh.vertices)
  faces = np.asarray(mesh.triangles)

  if labels is None:
    print("Mesh without Highlighting")
    # Create Plotly 3D mesh plot without highlithing
    fig = go.Figure(
            data=[
                go.Mesh3d(
                    x=vertices[:, 0], y=vertices[:, 1], z=vertices[:, 2],
                    i=faces[:, 0], j=faces[:, 1], k=faces[:, 2],
                    color=base_color,
                    opacity=1
                  )
            ])

  else:
    print("Mesh with Highlighting")
    # Handling coloring mesh
    kdtree = cKDTree(coordinates)
    # Trova l'indice del punto più vicino nella point cloud per ogni vertice della mesh
    _, idx = kdtree.query(vertices)
    # Colori dei vertici basati sui valori della label
    vertex_colors = labels[idx]

    # Create Plotly 3D mesh plot with highlithing
    fig = go.Figure(
        data=[
            go.Mesh3d(
                x=vertices[:, 0], y=vertices[:, 1], z=vertices[:, 2],
                i=faces[:, 0], j=faces[:, 1], k=faces[:, 2],
                intensity=vertex_colors,  # Colori basati sui valori interpolati
                colorscale = [
                  [0, base_color],  # Grey (Low density)
                  [1, highlithed_color]     # Yellow (High density)
                ],  # Mappa colori
                colorbar=dict(title='Density'),  # Barra colori
                opacity=1
        )])


  # Set plot titles, labels, and aspect ratio
  fig.update_layout(
      title= 'Highlithed Mesh' if labels is not None else '3D Mesh from Point Cloud',
      scene=dict(
          xaxis=dict(showgrid=False, showticklabels=False, showbackground=False, title=''),
          yaxis=dict(showgrid=False, showticklabels=False, showbackground=False, title=''),
          zaxis=dict(showgrid=False, showticklabels=False, showbackground=False, title=''),
          aspectmode='data'
      ),
  )

  fig.show()


# **From dataset to Point Cloud**

# **Point Cloud without Normals**

**Create Point Cloud**

In [9]:
# Coordinates of each point of the point cloud
coordinates = data[selected_obj_index]['full_shape']['coordinate']
# Densities associated to each point based on selected_label
labels = data[selected_obj_index]['full_shape']['label'][selected_label].flatten()
#Save coordinates file
np.savetxt("coordinates.txt", coordinates, fmt="%.6f", comments="")

# Point Cloud (without normals)
pcd = o3d.io.read_point_cloud("coordinates.txt", format='xyz')

**Visualizing point cloud with and without highlithing**

In [10]:
# Visualizing point cloud
show_pc(pcd)

# Visualizing point cloud based on selected label
show_pc(pcd, labels)

Point Cloud without Highlighting


Point Cloud with Highlighting


**Handling colored point cloud**

In [ ]:
'''
# Costruisce un KDTree per la ricerca dei vicini
pcd_tree = o3d.geometry.KDTreeFlann(pcd)

# Lista per memorizzare le distanze
distances = []

# Calcola la distanza al primo vicino per ogni punto
for i in range(len(pcd.points)):
    # Cerca i 2 punti più vicini (incluso se stesso)
    [_, idx, dists] = pcd_tree.search_knn_vector_3d(pcd.points[i], 2)
    # Salva la distanza del secondo vicino (esclude se stesso)
    distances.append(dists[1])

# Calcola la distanza media
mean_distance = np.mean(distances)
print("Mean distance:", mean_distance)'''

'\n# Costruisce un KDTree per la ricerca dei vicini\npcd_tree = o3d.geometry.KDTreeFlann(pcd)\n\n# Lista per memorizzare le distanze\ndistances = []\n\n# Calcola la distanza al primo vicino per ogni punto\nfor i in range(len(pcd.points)):\n    # Cerca i 2 punti più vicini (incluso se stesso)\n    [_, idx, dists] = pcd_tree.search_knn_vector_3d(pcd.points[i], 2)\n    # Salva la distanza del secondo vicino (esclude se stesso)\n    distances.append(dists[1])\n\n# Calcola la distanza media\nmean_distance = np.mean(distances)\nprint("Mean distance:", mean_distance)'

# **Point Cloud with normals**

**Adding normals to the point cloud**

In [11]:
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
#pcd.orient_normals_to_align_with_direction([0.0, 0.0, 1.0])
pcd.orient_normals_consistent_tangent_plane(30)

**Visualizing point cloud with normals**

In [13]:
# Coordinates and normals to numpy array
points = np.asarray(pcd.points)
normals = np.asarray(pcd.normals)

# Coordina il rendering
fig = go.Figure()

# Aggiungi la visualizzazione della Point Cloud
fig.add_trace(go.Scatter3d(
    x=points[:, 0],
    y=points[:, 1],
    z=points[:, 2],
    mode='markers',
    marker=dict(
        size=3,
        color=points[:, 2],
        colorscale='Viridis',
        opacity=0.7),
    name="Point Cloud"
))

for i in range(len(points)):  # Rimuove il campionamento, itera su tutti i punti
    x_start, y_start, z_start = points[i]
    x_end, y_end, z_end = points[i] + normals[i] * 0.1  # Scala la lunghezza delle normali
    fig.add_trace(go.Scatter3d(
        x=[x_start, x_end],
        y=[y_start, y_end],
        z=[z_start, z_end],
        mode='lines',
        line=dict(color='red', width=2),
        showlegend=False
    ))

fig.update_layout(
    title='Point Cloud with Normals',
    scene=dict(
        xaxis=dict(showgrid=False, showticklabels=False),
        yaxis=dict(showgrid=False, showticklabels=False),
        zaxis=dict(showgrid=False, showticklabels=False),
        aspectmode='data'
        ),
    showlegend=False
)

fig.show()


print("Number of points:", len(pcd.points))
print("Number of normals:", len(pcd.normals))


Number of points: 2048
Number of normals: 2048


# **Point Cloud to Mesh**

# **Method 1: Alpha shapes**

In [14]:
alpha = 0.09
print(f"alpha={alpha:.3f}")

alpha=0.090


In [15]:
mesh_from_pc = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
mesh_from_pc.compute_vertex_normals()

TriangleMesh with 1777 points and 3536 triangles.

In [16]:
'''
tetra_mesh, pt_map = o3d.geometry.TetraMesh.create_from_point_cloud(pcd)
for alpha in np.logspace(np.log10(0.5), np.log10(0.05), num=4):
    print(f"alpha={alpha:.3f}")
    mesh_from_pc = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(
        pcd, alpha, tetra_mesh, pt_map)
    mesh_from_pc.compute_vertex_normals()
'''


'\ntetra_mesh, pt_map = o3d.geometry.TetraMesh.create_from_point_cloud(pcd)\nfor alpha in np.logspace(np.log10(0.5), np.log10(0.05), num=4):\n    print(f"alpha={alpha:.3f}")\n    mesh_from_pc = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(\n        pcd, alpha, tetra_mesh, pt_map)\n    mesh_from_pc.compute_vertex_normals()\n'

# **Method 2: Ball pivoting**

In [18]:
from scipy.spatial import distance

def calcola_distanza_media(points):
    n = len(points)
    distanze = []
    for i in range(n):
        distanze_punti = distance.cdist([points[i]], points, 'euclidean')[0]
        distanze_punti = distanze_punti[distanze_punti > 0]  # escludiamo la distanza a se stessi
        distanza_media = np.mean(distanze_punti)
        distanze.append(distanza_media)

    distanza_media_totale = np.mean(distanze)
    return distanza_media_totale

#points = np.asarray(pcd.points)
distanza_media = calcola_distanza_media(pcd.points)
print("Distanza media tra i punti:", distanza_media)
raggio = distanza_media * 1.75
print("Raggio della sfera:", raggio)

Distanza media tra i punti: 0.6515046381448305
Raggio della sfera: 1.1401331167534534


In [19]:
radii = [0.0005, 0.005, 0.01, 0.02, 0.04, 0.08]
mesh_from_pc = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, o3d.utility.DoubleVector(radii), )

# **Method 3: Poisson surface reconstruction**

In [12]:
mesh_from_pc, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)

# **Results**

In [20]:
from scipy.spatial import cKDTree  # Per interpolazione dei colori tra i vertici

print(mesh_from_pc)
o3d.io.write_triangle_mesh("mesh_from_pc.obj", mesh_from_pc)
mesh = o3d.io.read_triangle_mesh('mesh_from_pc.obj')

# Mesh without highlithing
show_mesh(mesh)

# Mesh with highlithing
show_mesh(mesh, labels)

TriangleMesh with 2048 points and 4008 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
Mesh without Highlighting


Mesh with Highlighting


# **Save results**

In [ ]:
output_filename = f"highlithed_mesh_{selected_label}.ply"
o3d.io.write_triangle_mesh(output_filename, mesh)

True